# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

706948

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

-_>>


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

62

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['prouva', 'de', 'la', 'falsification', '.'], ['elle', 'l', "'", 'a', 'si', 'uicit', 'senti', 'qu', "'", 'elle', 'vient_d', "'", 'adresser', 'aux', 'membres', 'dc', 'la_cour', 'des_explications', 'pour', 'refuter', 'l', "'", 'argumentation', 'de', 'm', '"', 'b', '<', 't', "'", 'tolaud', '.'], ['<<', 'les', 'livres', 'lie', 'sont', 'pas', 'd', "'", 'accord', ',', '>>', 'me', 'dit', '-', 'on', '.'], ['quoi', 'd', "'", 'etonnant', 'a', 'cela', '?'], ['l', "'", 'un', 'contenait', '#>>', 'des', 'faits', ',', 'l', "'", 'aulre', 'des', 'impressions', '.'], ['or', ',', 'quand_je', '>>', 'jetais', 'mes', 'impressions', 'je_n', "'", 'attachais', 'aucune', 'impor', '-', '>>', 'tanca', 'aux', 'dates', '!'], ['>>', 'explication', 'bien', 'insuffisante', ':', '*', 'pourquoi', 'les', 'mettiez', '-', 'vous', '?'], ['mais', 'il', 'n', "'", 'y', 'a', 'pas', 'que', 'la_date', 'de', 'fausse', ';', 'ce', 'qui', 'se_trouve', 'sur', 'e', 'carne', 't', 'est', 'date', 'a', 'chaque', 'ligne', 'par', 'certains'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [85]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=1, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 19.2 s
Wall time: 8.36 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [86]:
outfile = f"../data/sents.txt"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [87]:
model = Word2Vec.load("../data/sents.txt")

### Imprimer le vecteur d'un terme

In [88]:
model.wv["vaccin"]

array([-0.00155834, -0.71099657,  0.5927693 ,  0.41718817,  0.29334536,
       -0.34502226,  0.36971065,  0.3293791 ,  0.0253643 , -0.35661077,
        0.27195376, -0.14002466, -0.08990686, -0.3269614 ,  0.24489412,
       -0.03300734, -0.46130705,  0.16409607,  0.82280767, -0.01309614,
        1.0528188 ,  1.0611748 ,  0.2980314 ,  0.0690366 , -0.07991336,
        0.47552693, -0.75191414, -0.14148535, -0.32444546,  0.24854971,
        0.04758052,  0.79001117], dtype=float32)

### Calculer la similarité entre deux termes

In [93]:
model.wv.similarity("vaccination", "dogme")

0.17758182

### Chercher les mots les plus proches d'un terme donné

In [104]:
model.wv.most_similar("vaccines", topn=40)

[('faibles', 0.8988722562789917),
 ('moutons', 0.8928266763687134),
 ('ames', 0.8921120762825012),
 ('restent', 0.8908241391181946),
 ('bien_portants', 0.8886749148368835),
 ('insignifiantes', 0.8880680799484253),
 ('vaccinees', 0.8766351342201233),
 ('valeurs_industrielles', 0.8734846115112305),
 ('producteurs', 0.8683286905288696),
 ('perdent', 0.8682953119277954),
 ('la_consommation', 0.8676751852035522),
 ('expediees', 0.8670964241027832),
 ('morts', 0.8620927929878235),
 ('abondantes', 0.8567662239074707),
 ('dommages', 0.8548779487609863),
 ('ces_derniers', 0.8528593182563782),
 ('inocules', 0.8527267575263977),
 ('acheteurs', 0.8522994518280029),
 ('les_affaires', 0.8521357774734497),
 ('fermement', 0.848473310470581),
 ('transactions', 0.8472214341163635),
 ('provenances', 0.843711793422699),
 ('poules', 0.8425905704498291),
 ('speculateurs', 0.8396398425102234),
 ('concurrents', 0.8394684195518494),
 ('precedents', 0.8391222953796387),
 ('sont_restes', 0.8386133313179016),
 ('

### Faire des recherches complexes à travers l'espace vectoriel

In [20]:
print(model.wv.most_similar(positive=['vaccin', 'londres'], negative=['belgique']))

[('berlin', 0.807077944278717), ('rome', 0.7655918598175049), ('tokio', 0.7614746689796448), ('moscou', 0.7519477605819702), ('teheran', 0.7468174695968628), ('datee_du', 0.745754063129425), ('londres_londres', 0.7429144382476807), ('vienne', 0.7417282462120056), ('strasbourg', 0.7280466556549072), ('novembre', 0.7275460362434387)]
